In [7]:
!pip install pandas matplotlib
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

DB_PATH = "/content/drive/MyDrive/Chicago311.db"  # Updated path for mounted Google Drive
OUT_DIR = "output"
os.makedirs(OUT_DIR, exist_ok=True)

def load_df(query: str) -> pd.DataFrame:
    with sqlite3.connect(DB_PATH) as conn:
        return pd.read_sql_query(query, conn)

def savefig(name: str):
    path = os.path.join(OUT_DIR, name)
    plt.tight_layout()
    plt.savefig(path, dpi=200)
    plt.close()
    print(f"Saved: {path}")

# 1) Top 10 request types
df_top_types = load_df("""
SELECT sr_type, COUNT(*) AS request_count
FROM service_requests
GROUP BY sr_type
ORDER BY request_count DESC
LIMIT 10;
""")

plt.figure()
plt.barh(df_top_types["sr_type"][::-1], df_top_types["request_count"][::-1])
plt.title("Top 10 Chicago 311 Request Types (Count)")
plt.xlabel("Requests")
savefig("01_top_request_types.png")

# 2) Resolution time histogram (trim extreme outliers for readability)
df_res = load_df("""
SELECT resolution_hours
FROM service_requests
WHERE resolution_hours IS NOT NULL;
""")

# Trim to 99th percentile for the main histogram
p99 = df_res["resolution_hours"].quantile(0.99)
df_trim = df_res[df_res["resolution_hours"] <= p99]

plt.figure()
plt.hist(df_trim["resolution_hours"], bins=50)
plt.title("Resolution Time Distribution (trimmed at 99th percentile)")
plt.xlabel("Resolution hours")
plt.ylabel("Count")
savefig("02_resolution_hist_trimmed.png")

# Optional: log-scale histogram (helps with skew)
plt.figure()
plt.hist(df_res["resolution_hours"], bins=60)
plt.yscale("log")
plt.title("Resolution Time Distribution (log y-scale)")
plt.xlabel("Resolution hours")
plt.ylabel("Count (log scale)")
savefig("03_resolution_hist_log.png")

# 3) Department performance (avg resolution hours)
df_dept = load_df("""
SELECT owner_department,
       COUNT(*) AS n,
       ROUND(AVG(resolution_hours), 2) AS avg_hours
FROM service_requests
WHERE resolution_hours IS NOT NULL
GROUP BY owner_department
ORDER BY avg_hours DESC;
""")

plt.figure()
plt.barh(df_dept["owner_department"][::-1], df_dept["avg_hours"][::-1])
plt.title("Average Resolution Hours by Department")
plt.xlabel("Avg resolution hours")
savefig("04_avg_resolution_by_department.png")

# 4) Resolution buckets
df_buckets = load_df("""
SELECT
  CASE
    WHEN resolution_hours IS NULL THEN 'Open'
    WHEN resolution_hours = 0 THEN '0 hours'
    WHEN resolution_hours <= 24 THEN '0–24 hours'
    WHEN resolution_hours <= 72 THEN '1–3 days'
    WHEN resolution_hours <= 168 THEN '3–7 days'
    ELSE '7+ days'
  END AS resolution_bucket,
  COUNT(*) AS cnt
FROM service_requests
GROUP BY resolution_bucket
ORDER BY cnt DESC;
""")

plt.figure()
plt.bar(df_buckets["resolution_bucket"], df_buckets["cnt"])
plt.title("Resolution Time Buckets")
plt.xlabel("Bucket")
plt.ylabel("Count")
plt.xticks(rotation=25, ha="right")
savefig("05_resolution_buckets.png")

# 5) Day-of-week x Hour heatmap
df_heat = load_df("""
SELECT created_day_of_week, created_hour, COUNT(*) AS requests
FROM service_requests
GROUP BY created_day_of_week, created_hour
ORDER BY created_day_of_week, created_hour;
""")

# Pivot into matrix
heat = df_heat.pivot(index="created_day_of_week", columns="created_hour", values="requests").fillna(0)

plt.figure()
plt.imshow(heat.values, aspect="auto")
plt.title("Requests Heatmap: Day of Week x Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Day of Week")
plt.xticks(range(len(heat.columns)), heat.columns, rotation=90)
plt.yticks(range(len(heat.index)), heat.index)
plt.colorbar(label="Requests")
savefig("06_requests_heatmap.png")

# 6) Optional: map-ish scatter plot (lat/long colored by bucket)
df_map = load_df("""
SELECT latitude, longitude,
  CASE
    WHEN resolution_hours IS NULL THEN 'Open'
    WHEN resolution_hours = 0 THEN '0 hours'
    WHEN resolution_hours <= 24 THEN '0–24 hours'
    WHEN resolution_hours <= 168 THEN '1–7 days'
    ELSE '7+ days'
  END AS bucket
FROM service_requests
WHERE latitude IS NOT NULL AND longitude IS NOT NULL;
""")

# Sample for speed/readability
df_map = df_map.sample(min(len(df_map), 5000), random_state=42)

plt.figure()
for b in df_map["bucket"].unique():
    sub = df_map[df_map["bucket"] == b]
    plt.scatter(sub["longitude"], sub["latitude"], s=4, label=b)

plt.title("Chicago 311 Requests (sample) by Resolution Bucket")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend(markerscale=3, fontsize=8)
savefig("07_map_scatter_bucket.png")

# Inspect database for tables
print(load_df("SELECT name FROM sqlite_master WHERE type='table';"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved: output/01_top_request_types.png
Saved: output/02_resolution_hist_trimmed.png
Saved: output/03_resolution_hist_log.png
Saved: output/04_avg_resolution_by_department.png
Saved: output/05_resolution_buckets.png
Saved: output/06_requests_heatmap.png
Saved: output/07_map_scatter_bucket.png
               name
0  service_requests
